# Ice Sheet Emulator Demo

### Installation

To install, run this is your terminal. This will create an environment, download ISE, and activate a jupyter notebook instance. Open up this notebook to run the model.  

```conda create -n ise -y```  
```conda activate ise```  
```conda install nb_conda ipykernel -y```  
```pip install git+https://github.com/Brown-SciML/ise```  
```jupyter notebook```  

In [1]:
# pip install git+https://github.com/Brown-SciML/ise --force-reinstall

In [2]:
import numpy as np

from ise.models.hybrid import HybridEmulator, DeepEnsemble, NormalizingFlow
from sklearn.preprocessing import StandardScaler

### Synthetic Data and Scaling

In [3]:
X = np.array([[1,2,3], [4,5,6], [7,8,9]], dtype=int)
y = np.array([[1], [2], [3],], dtype=int)

scaler_y = StandardScaler().fit(y)

### Create emulator Model
Create the model with the Deep Ensemble as the Predictor and the Normalizing Flow as the uncertainty quantifier.

In [4]:
input_shape = X.shape[1]
output_shape = y.shape[1]
num_ensemble_members = 2

predictor = DeepEnsemble(num_predictors=num_ensemble_members, forcing_size=input_shape, sle_size=output_shape)
uncertainty_quantifier = NormalizingFlow(forcing_size=input_shape, sle_size=output_shape)
emulator = HybridEmulator(deep_ensemble=predictor, normalizing_flow=uncertainty_quantifier)

TypeError: hidden_size should be of type int, got: int64

In [ ]:
emulator.fit(X, y, nf_epochs=10, de_epochs=15)

### Predict on unseen data

In [ ]:
predictions, uncertainties = emulator.predict(np.array([[10, 11, 12]]), output_scaler=scaler_y)

In [ ]:
predictions

In [ ]:
uncertainties